<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2021<br>
            Dr. Eric Van Dusen <br>
        Alan Liang, Umar Maniku <br>
</table>

# Lecture 10: Macroeconomics Policy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datascience import * 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline

In [ ]:
gdp = pd.read_csv("real_gdp_over_time.csv")
gdp_growth = pd.read_csv("real_growth_over_time.csv")
inflation = pd.read_csv("inflation_over_time.csv")
unemployment = pd.read_csv("unemployment_over_time.csv")
deficit = pd.read_csv("deficit_over_time.csv")
ffr = pd.read_csv("fedfunds.csv")
potential_gdp = pd.read_csv("potential_gdp.csv")
unemployment_gap = pd.read_csv("nrou.csv")

gdp = gdp.set_index("DATE")
gdp_growth = gdp_growth.set_index("DATE")
inflation = inflation.set_index("DATE")
unemployment = unemployment.set_index("DATE")
deficit = deficit.set_index("DATE")
ffr = ffr.set_index("DATE")
unemployment_gap = unemployment_gap.set_index("DATE")
potential_gdp = potential_gdp.set_index("DATE")

macroeconomics = gdp.merge(gdp_growth, left_index = True, right_index = True)\
                    .merge(inflation, left_index = True, right_index = True)\
                    .merge(unemployment, left_index = True, right_index = True)\
                    .merge(ffr, left_index = True, right_index = True)

potential_gdp = potential_gdp[potential_gdp.index <= max(macroeconomics.index)]

In [ ]:
macroeconomics.head()

## Time series: 1958 to present

In [ ]:
fig, ax = plt.subplots(6, 1)
for axe, col in zip(ax.flatten(), list(macroeconomics.columns) + ["DEFICIT"]):
    if col == "DEFICIT":
        axe.plot(deficit.index, deficit.DEFICIT)
        axe.set_xticks(np.arange(0, deficit.shape[0], 5))
        axe.set_xticklabels(deficit.index[np.arange(0, deficit.shape[0], 5)], rotation=45)
        axe.set_title(col)
        axe.grid()
        axe.axhline(color="black", linewidth=3)
    else:
        axe.plot(macroeconomics.index, macroeconomics[col])
        axe.set_xticks(np.arange(0, macroeconomics.shape[0], 20))
        axe.set_xticklabels(macroeconomics.index[np.arange(0, macroeconomics.shape[0], 20)], rotation=45)
        axe.set_title(col)
        axe.grid()
        axe.axhline(color="black", linewidth=3)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.set_size_inches(16, 60)

## Macroeconomic Indicators during the pandemic

In [ ]:
covid = macroeconomics[macroeconomics.index >= "2019-01-01"]
covid_deficit = deficit[deficit.index >= "2019-01-01"]
covid_potential_gdp = potential_gdp[potential_gdp.index >= "2019-01-01"]
covid_ffr = ffr[ffr.index >= "2019-01-01"]

In [ ]:
fig, ax = plt.subplots(6, 1)
for axe, col in zip(ax.flatten(), list(covid.columns) + ["DEFICIT"]):
    if col == "DEFICIT":
        axe.plot(covid_deficit.index, covid_deficit.DEFICIT)
        axe.set_xticks(covid_deficit.index)
        axe.set_xticklabels(covid_deficit.index, rotation=45)
        axe.set_title(col)
        axe.grid()
        axe.axhline(color="black", linewidth=3)
    else:
        axe.plot(covid.index, covid[col])
        axe.set_xticks(covid.index)
        axe.set_xticklabels(covid.index, rotation=45)
        axe.set_title(col)
        axe.grid()
        axe.axhline(color="black", linewidth=3)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
fig.set_size_inches(16, 60)

## Federal Funds Rate and The Taylor Rule: 1958 to Present

### Taylor Rule:
$$
r_{\text{predicted}} = r_{\text{target}} + \pi + \alpha \left ( \pi - \pi_{\text{target}} \right ) + \beta \left ( \dfrac{Y - Y^*}{Y^*} \right )
$$

In [ ]:
def taylor_rule(interest_rate_target, inflation, inflation_target, output, potential_output, alpha=0.5, beta=0.5):
    return interest_rate_target + inflation + alpha * (inflation - inflation_target) + \
            beta * ((output - potential_output) / potential_output) * 100

In [ ]:
def taylor_rule_plotter(arr, ffr_arr=None, label=None):
    plt.figure(figsize=(17, 9))
    if ffr_arr is not None:
        plt.plot(ffr_arr, label="FFR")
        plt.xticks(np.arange(0, ffr_arr.shape[0]), rotation=60)
    else:
        plt.plot(macroeconomics.loc[:, "FEDFUNDS"], label="FFR")
        plt.xticks(np.arange(0, ffr.shape[0], 20), rotation=60)
    plt.plot(arr, label=label)
    plt.legend()
    plt.grid()
    plt.axhline(color="black", linewidth=2)
    plt.show();

In [ ]:
taylor_rule_preds = taylor_rule(2, macroeconomics.loc[:, "INFLATION"], 2, 
                                macroeconomics.loc[:, "GDP"], potential_gdp.loc[:, "GDPPOT"])

taylor_rule_plotter(taylor_rule_preds, label=r"Taylor Rule FFR with $\alpha$=0.5 and $\beta$=0.5")

## FFR and The Taylor Rule during the pandemic

In [ ]:
taylor_rule_preds_covid = taylor_rule(2, covid.loc[:, "INFLATION"], 2, 
                                covid.loc[:, "GDP"], covid_potential_gdp.loc[:, "GDPPOT"])

taylor_rule_plotter(taylor_rule_preds_covid, ffr_arr=covid.loc[:, "FEDFUNDS"], 
                    label=r"Taylor Rule FFR with $\alpha$=0.5 and $\beta$=0.5")

In [ ]:
def taylor_rmse(alpha, beta):
    taylor_interest_rate = np.array(taylor_rule(2, macroeconomics.loc[:, "INFLATION"], 2, 
                                                macroeconomics.loc[:, "GDP"], potential_gdp.loc[:, "GDPPOT"], 
                                                alpha = alpha, beta = beta))
    actual_interest_rate = np.array(macroeconomics.FEDFUNDS)
    return np.mean(((taylor_interest_rate - actual_interest_rate) ** 2))

In [ ]:
def taylor_widget(alpha, beta):
    rmse = taylor_rmse(alpha, beta)
    print(f"RMSE: {rmse}")
    taylor_rule_preds = taylor_rule(2, macroeconomics.loc[:, "INFLATION"], 2, 
                                macroeconomics.loc[:, "GDP"], potential_gdp.loc[:, "GDPPOT"], 
                                    alpha=alpha, beta=beta)
    taylor_rule_plotter(taylor_rule_preds, 
                        label=rf"Taylor Rule FFR with $\alpha$={alpha} and $\beta$={beta}");

In [ ]:
alpha_widget = widgets.FloatSlider(min=-1, max=2, step=0.01, value=0.5)
beta_widget = widgets.FloatSlider(min=-1, max=2, step=0.01, value=0.5)
interact(taylor_widget, alpha=alpha_widget, beta=beta_widget)

## Optimizing for $\alpha$, $\beta$

In [ ]:
alpha, beta = minimize(taylor_rmse)
print(f"RMSE using optimized parameters: {taylor_rmse(alpha, beta)}")
print(f"alpha: {alpha}, beta: {beta}")

In [ ]:
taylor_rule_preds = taylor_rule(2, macroeconomics.loc[:, "INFLATION"], 2, 
                                macroeconomics.loc[:, "GDP"], potential_gdp.loc[:, "GDPPOT"], 
                                alpha=alpha, beta=beta)
taylor_rule_preds_basic = taylor_rule(2, macroeconomics.loc[:, "INFLATION"], 2, 
                                      macroeconomics.loc[:, "GDP"], potential_gdp.loc[:, "GDPPOT"], 
                                      alpha=0.5, beta=0.5)

plt.figure(figsize=(15, 8))
plt.plot(ffr, label="FFR")
plt.plot(taylor_rule_preds, label=fr"Taylor Rule FFR optimized with $\alpha={np.round(alpha, 2)}$ and $\beta={np.round(beta, 2)}$")
plt.plot(taylor_rule_preds_basic, label=r"Taylor Rule FFR with $\alpha$=0.5 and $\beta$=0.5")
plt.xticks(rotation=45)
plt.xticks(np.arange(0, ffr.shape[0], 20), rotation=60)
plt.legend()
plt.grid()
plt.title("FFR vs. Taylor Rule FFR")
plt.show();